In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import cross_val_score

# 학습 데이터 로딩
titanic_data = pd.read_csv('./titanic_train.csv')

# 'Cabin' 열 제거
titanic_data = titanic_data.drop(columns='Cabin', axis=1)

# 'Age' 열의 결측치를 평균값으로 대체
titanic_data['Age'].fillna(titanic_data['Age'].mean(), inplace=True)

# 'Embarked' 열의 결측치를 최빈값으로 대체
titanic_data['Embarked'].fillna(titanic_data['Embarked'].mode()[0], inplace=True)

# 'Survived'를 문자열로 변환
titanic_data['Survived'] = titanic_data['Survived'].astype(str)

# 'Embarked' 열의 문자열을 숫자로 변환
titanic_data.replace({'Sex': {'male': 0, 'female': 1}, 'Embarked': {'S': 0, 'C': 1, 'Q': 2}, 'Survived': {'0': 0, '1': 1}}, inplace=True)

# 'Name', 'Ticket', 'PassengerId' 열 제거
titanic_data = titanic_data.drop(columns=['Name', 'Ticket', 'PassengerId'], axis=1)

# 독립 변수(X)와 종속 변수(Y) 설정
X = titanic_data.drop(columns=['Survived'], axis=1)
Y = titanic_data['Survived']

# 학습 데이터와 테스트 데이터로 분할
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=2)

# XGBoost 모델 초기화 및 학습
xgb_model = XGBClassifier(objective='binary:logistic', random_state=2)
xgb_model.fit(X_train, Y_train)

# 학습 데이터 정확도 출력
X_train_prediction_xgb = xgb_model.predict(X_train)
training_data_accuracy_xgb = accuracy_score(Y_train, X_train_prediction_xgb)
print('Accuracy score of training data (XGBoost): ', training_data_accuracy_xgb)

# 테스트 데이터 정확도 출력
X_test_prediction_xgb = xgb_model.predict(X_test)
test_data_accuracy_xgb = accuracy_score(Y_test, X_test_prediction_xgb)
print('Accuracy score of test data (XGBoost): ', test_data_accuracy_xgb)

# 교차 검증을 통한 정확도 확인
cross_val_scores_xgb = cross_val_score(xgb_model, X, Y, cv=5)
print('Cross-validated accuracy (XGBoost):', np.mean(cross_val_scores_xgb))

# 테스트 데이터 로딩
titanic_test = pd.read_csv('./titanic_test.csv')

# 'Cabin' 열 제거
titanic_test = titanic_test.drop(columns='Cabin', axis=1)

# 'Age' 열의 결측치를 평균값으로 대체
titanic_test['Age'].fillna(titanic_test['Age'].mean(), inplace=True)

# 'Fare' 열의 결측치를 평균값으로 대체
titanic_test['Fare'].fillna(titanic_test['Fare'].mean(), inplace=True)

# 'Embarked' 열의 결측치를 최빈값으로 대체
titanic_test['Embarked'].fillna(titanic_test['Embarked'].mode()[0], inplace=True)

# 'Embarked' 열의 문자열을 숫자로 변환
titanic_test.replace({'Sex': {'male': 0, 'female': 1}, 'Embarked': {'S': 0, 'C': 1, 'Q': 2}}, inplace=True)

# 'Name', 'Ticket', 'PassengerId' 열 제거
titanic_test = titanic_test.drop(columns=['Name', 'Ticket', 'PassengerId'], axis=1)

# 테스트 데이터에 대한 예측 수행
test_predictions_xgb = xgb_model.predict(titanic_test)

# 예측 결과를 데이터프레임으로 변환
submission_xgb = pd.DataFrame({'PassengerId': pd.read_csv('./titanic_test.csv')['PassengerId'], 'Survived': test_predictions_xgb})

# CSV 파일로 저장
# CSV 파일로 저장
submission_xgb.to_csv('titanic_submission_xgb.csv', index=False)


Accuracy score of training data (XGBoost):  0.973314606741573
Accuracy score of test data (XGBoost):  0.7988826815642458
Cross-validated accuracy (XGBoost): 0.8204569706860838
